In [7]:
import numpy as np
import pandas as pd
import adaboost
import utils
import noising
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

%matplotlib inline

data = pd.read_csv("DataSet/liver-disorder/bupa.csv", header=None)
Y_index = 0
indexes_to_add_feature_noise = [2, 3, 4, 5, 6]
max_stds = data[indexes_to_add_feature_noise].describe().ix["std"] * 0.1
pPos, pNeg = 0, 0
number_of_tries = 10

Ts = [10 * i for i in range(1, 21)]
Cs = [10 ** i for i in range(-2, 3)]

data.describe()

,0,1,2,3,4,5,6
count,345.000000,345.000000,345.000000,345.000000,345.000000,345.000000,345.000000
mean,0.420290,90.159420,69.869565,30.405797,24.643478,38.284058,3.455072
std,0.494322,4.448096,18.347670,19.512309,10.064494,39.254616,3.337835
min,0.000000,65.000000,23.000000,4.000000,5.000000,5.000000,0.000000
25%,0.000000,87.000000,57.000000,19.000000,19.000000,15.000000,0.500000
50%,0.000000,90.000000,67.000000,26.000000,23.000000,25.000000,3.000000
75%,1.000000,93.000000,80.000000,34.000000,27.000000,46.000000,6.000000
max,1.000000,103.000000,138.000000,155.000000,82.000000,297.000000,20.000000


In [8]:
def TryOuts(indexes_to_add_feature_noise, max_stds, pPos, pNeg):
    reload(noising)
    reload(utils)

    Val_error = np.empty(0)
    for p in range(number_of_tries):
        # Add noise on features (before scaling)
        noisy_data = data
        for idx in indexes_to_add_feature_noise:
            std = max_stds.ix[idx]
            if std > 0:
                noisy_data[idx] = noising.addNormalNoise(noisy_data[idx], avg, std)
        # Prepare dataset
        noisy_data["Y"] = np.where(noisy_data[Y_index] == 0, -1, 1)
        noisy_data = noisy_data.drop(Y_index, 1)
        noisy_data[noisy_data.drop("Y", 1).columns.values] = scale(noisy_data[noisy_data.drop("Y", 1).columns.values])
        # Split train / test
        X_train, X_test, y_train, y_test = train_test_split(noisy_data.drop("Y", 1), noisy_data["Y"], test_size=0.33)
        # Add noise on labels (On training set ! otherwise doesn't make sense)
        y_train = noising.switchLabels(y_train, pPos, pNeg)
        # Cross Validate
        best_T, cross_validate_accuracy_training = utils.cross_validate_adaboost_T(X_train, y_train, Ts, None, None, 0, False)
        # Val. Error
        my_adaboost = adaboost.Adaboost(best_T)
        my_adaboost.fit(X_train, y_train)
        error_val = 1 - my_adaboost.score(X_test, y_test)
        Val_error = np.append(Val_error, error_val)
        #print " Round : {0}, Best T : {1}, train error : {2}, val. error : {3}".format(p + 1, best_T, 1 - cross_validate_accuracy_training, error_val)

    print("Adaboost Avg. val error : {0}, Std. : {1}".format(Val_error.mean(), Val_error.std()))

    Val_error = np.empty(0)
    for p in range(number_of_tries):
        # Add noise on features (before scaling)
        noisy_data = data
        for idx in indexes_to_add_feature_noise:
            std = max_stds.ix[idx]
            if std > 0:
                noisy_data[idx] = noising.addNormalNoise(noisy_data[idx], avg, std)
        # Prepare dataset
        noisy_data["Y"] = np.where(noisy_data[Y_index] == 0, -1, 1)
        noisy_data = noisy_data.drop(Y_index, 1)
        noisy_data[noisy_data.drop("Y", 1).columns.values] = scale(noisy_data[noisy_data.drop("Y", 1).columns.values])
        # Split train / test
        X_train, X_test, y_train, y_test = train_test_split(noisy_data.drop("Y", 1), noisy_data["Y"], test_size=0.33)
        # Add noise on labels (On training set ! otherwise doesn't make sense)
        y_train = noising.switchLabels(y_train, pPos, pNeg)
        # Cross Validate
        best_C, best_T, cross_validate_accuracy_training = utils.cross_validate_adaboost_T_C(X_train, y_train, Ts, 2, Cs, 0, False)
        # Val. Error
        my_adaboost = adaboost.Adaboost(best_T, 2, best_C, 0)
        my_adaboost.fit(X_train, y_train)
        error_val = 1 - my_adaboost.score(X_test, y_test)
        Val_error = np.append(Val_error, error_val)
        #print " Round : {0}, Best T / C : {1}, {2}, train error : {3}, val. error : {4}".format(p + 1, best_T, best_C, 1 - cross_validate_accuracy_training, error_val)

    print("L1 regulariazed Adaboost (paper) Avg. val error : {0}, Std. : {1}".format(Val_error.mean(), Val_error.std()))

    Val_error = np.empty(0)
    for p in range(number_of_tries):
        # Add noise on features (before scaling)
        noisy_data = data
        for idx in indexes_to_add_feature_noise:
            std = max_stds.ix[idx]
            if std > 0:
                noisy_data[idx] = noising.addNormalNoise(noisy_data[idx], avg, std)
        # Prepare dataset
        noisy_data["Y"] = np.where(noisy_data[Y_index] == 0, -1, 1)
        noisy_data = noisy_data.drop(Y_index, 1)
        noisy_data[noisy_data.drop("Y", 1).columns.values] = scale(noisy_data[noisy_data.drop("Y", 1).columns.values])
        # Split train / test
        X_train, X_test, y_train, y_test = train_test_split(noisy_data.drop("Y", 1), noisy_data["Y"], test_size=0.33)
        # Add noise on labels (On training set ! otherwise doesn't make sense)
        y_train = noising.switchLabels(y_train, pPos, pNeg)
        # Cross Validate
        best_C, best_T, cross_validate_accuracy_training = utils.cross_validate_adaboost_T_C(X_train, y_train, Ts, 2, Cs, 1, False)
        # Val. Error
        my_adaboost = adaboost.Adaboost(best_T, 2, best_C, 1)
        my_adaboost.fit(X_train, y_train)
        error_val = 1 - my_adaboost.score(X_test, y_test)
        Val_error = np.append(Val_error, error_val)
        #print " Round : {0}, Best T : {1}, train error : {2}, val. error : {3}".format(p + 1, best_T, 1 - cross_validate_accuracy_training, error_val)

    print("Adaboost v1 Avg. val error : {0}, Std. : {1}".format(Val_error.mean(), Val_error.std()))

    Val_error = np.empty(0)
    for p in range(number_of_tries):
        # Add noise on features (before scaling)
        noisy_data = data
        for idx in indexes_to_add_feature_noise:
            std = max_stds.ix[idx]
            if std > 0:
                noisy_data[idx] = noising.addNormalNoise(noisy_data[idx], avg, std)
        # Prepare dataset
        noisy_data["Y"] = np.where(noisy_data[Y_index] == 0, -1, 1)
        noisy_data = noisy_data.drop(Y_index, 1)
        noisy_data[noisy_data.drop("Y", 1).columns.values] = scale(noisy_data[noisy_data.drop("Y", 1).columns.values])
        # Split train / test
        X_train, X_test, y_train, y_test = train_test_split(noisy_data.drop("Y", 1), noisy_data["Y"], test_size=0.33)
        # Add noise on labels (On training set ! otherwise doesn't make sense)
        y_train = noising.switchLabels(y_train, pPos, pNeg)
        # Cross Validate
        best_C, best_T, cross_validate_accuracy_training = utils.cross_validate_adaboost_T_C(X_train, y_train, Ts, 3, Cs, 1, False)
        # Val. Error
        my_adaboost = adaboost.Adaboost(best_T, 3, best_C, 1)
        my_adaboost.fit(X_train, y_train)
        error_val = 1 - my_adaboost.score(X_test, y_test)
        Val_error = np.append(Val_error, error_val)
        #print " Round : {0}, Best T : {1}, train error : {2}, val. error : {3}".format(p + 1, best_T, 1 - cross_validate_accuracy_training, error_val)

    print("Adaboost v1 pow 3Avg. val error : {0}, Std. : {1}".format(Val_error.mean(), Val_error.std()))

In [9]:
pPosMax, pNegMax = 0, 0
reload(utils)
reload(adaboost)

print "Noise level : 0 % of maximum"
noiseLevel = 0
TryOuts(indexes_to_add_feature_noise, max_stds * noiseLevel, pPosMax * noiseLevel, pNegMax * noiseLevel)

print "Noise level : 50 % of maximum"
noiseLevel = 0.5
TryOuts(indexes_to_add_feature_noise, max_stds * noiseLevel, pPosMax * noiseLevel, pNegMax * noiseLevel)

print "Noise level : 100 % of maximum"
noiseLevel = 1
TryOuts(indexes_to_add_feature_noise, max_stds * noiseLevel, pPosMax * noiseLevel, pNegMax * noiseLevel)

Noise level : 0 % of maximum
Adaboost Avg. val error : 0.305263157895, Std. : 0.0524558469745
 The cross validation is not effective, best C or T found on border
 The cross validation is not effective, best C or T found on border
 The cross validation is not effective, best C or T found on border
 The cross validation is not effective, best C or T found on border
 The cross validation is not effective, best C or T found on border
 The cross validation is not effective, best C or T found on border
 The cross validation is not effective, best C or T found on border
L1 regulariazed Adaboost (paper) Avg. val error : 0.278947368421, Std. : 0.0356964735954
 The cross validation is not effective, best C or T found on border
 The cross validation is not effective, best C or T found on border
 The cross validation is not effective, best C or T found on border
Adaboost v1 Avg. val error : 0.275438596491, Std. : 0.0415534009906
 The cross validation is not effective, best C or T found on border
 

NameError: global name 'avg' is not defined